### Goal 3, Question 1:
For entity type __Individual__, what is the "normal basket" of __procedures__ for each **provider type**?

#### Steps:
    1. read in csv and select rows that are ['Entity Type of the Provider'] == 'I']
    2. select necessary columns, for clarity
    3. Create query that returns top 15 procedures for given provider type

In [2]:
import pandas as pd
import pickle

#### read csv, select rows with Entity Type 'I' 

In [3]:
%%time

individual_providers_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         chunksize = 1000):
    individual_providers_rows.append(chunk[chunk['Entity Type of the Provider'] == 'I'])
    
               
                
individual_type_providers = pd.concat(individual_providers_rows, ignore_index=True)

Wall time: 2min 31s


#### Too many columns for this question...

In [4]:
individual_type_providers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 26 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Last Name/Organization Name of the Provider               object 
 2   First Name of the Provider                                object 
 3   Middle Initial of the Provider                            object 
 4   Credentials of the Provider                               object 
 5   Gender of the Provider                                    object 
 6   Entity Type of the Provider                               object 
 7   Street Address 1 of the Provider                          object 
 8   Street Address 2 of the Provider                          object 
 9   City of the Provider                                      object 
 10  Zip Code of the Provider      

#### select columns:
    - National Provider Identifier
    - Entity Type of the Provider
    - Provider Type
    - Place of Service
    - HCPCS Code
    - HCPCS Description
    - Number of Services
    - Number of Medicare Beneficiaries
    - Number of Distinct Medicare Beneficiary/Per Day Services

In [6]:
individual_type_providers_reduced_columns = individual_type_providers[['National Provider Identifier','Entity Type of the Provider',
                                                             'Provider Type','Place of Service','HCPCS Code',
                                                             'HCPCS Description','Number of Services',
                                                             'Number of Medicare Beneficiaries',
                                                             'Number of Distinct Medicare Beneficiary/Per Day Services']]

In [7]:
individual_type_providers_reduced_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9416125 entries, 0 to 9416124
Data columns (total 9 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   National Provider Identifier                              int64  
 1   Entity Type of the Provider                               object 
 2   Provider Type                                             object 
 3   Place of Service                                          object 
 4   HCPCS Code                                                object 
 5   HCPCS Description                                         object 
 6   Number of Services                                        float64
 7   Number of Medicare Beneficiaries                          int64  
 8   Number of Distinct Medicare Beneficiary/Per Day Services  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 646.6+ MB


In [10]:
individual_type_providers_reduced_columns['Provider Type'].nunique()

88

#### Diagnostic Radiology has the most rows, lets start by isolating that provider type...

In [17]:
individual_type_providers_reduced_columns['Provider Type'].value_counts()

Diagnostic Radiology                   1241400
Internal Medicine                      1118171
Family Practice                         969268
Nurse Practitioner                      560219
Cardiology                              445088
                                        ...   
Ambulance Service Provider                  42
Unknown Supplier/Provider Specialty         17
All Other Suppliers                         11
Slide Preparation Facility                   2
Medical Toxicology                           2
Name: Provider Type, Length: 88, dtype: int64

In [22]:
individual_type_providers_reduced_columns.loc[individual_type_providers_reduced_columns
                                              ['Provider Type'] == 'Diagnostic Radiology']


,National Provider Identifier,Entity Type of the Provider,Provider Type,Place of Service,HCPCS Code,HCPCS Description,Number of Services,Number of Medicare Beneficiaries,Number of Distinct Medicare Beneficiary/Per Day Services
348,1003002148,I,Diagnostic Radiology,F,70450,CT scan head or brain,156.0,151,156
349,1003002148,I,Diagnostic Radiology,F,70486,CT scan of face,23.0,23,23
350,1003002148,I,Diagnostic Radiology,F,70551,MRI scan brain,63.0,63,63
351,1003002148,I,Diagnostic Radiology,F,70553,MRI scan of brain before and after contrast,34.0,33,34
352,1003002148,I,Diagnostic Radiology,F,71010,"X-ray of chest, 1 view, front",437.0,363,423
...,...,...,...,...,...,...,...,...,...
9415612,1992994487,I,Diagnostic Radiology,F,72125,CT scan of upper spine,68.0,68,68
9415613,1992994487,I,Diagnostic Radiology,F,72170,"X-ray of pelvis, 1 or 2 views",11.0,11,11
9415614,1992994487,I,Diagnostic Radiology,F,74022,Imaging of abdomen and chest,12.0,12,12
9415615,1992994487,I,Diagnostic Radiology,F,74176,CT scan of abdomen and pelvis,45.0,45,45


#### Next, we need to do a value count of each HCPCS for radiology...